In [1]:
# needed to load the REBEL model
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
import math
import torch

# wrapper for wikipedia API
import wikipedia

# scraping of web articles
from newspaper import Article
# google news scraping
from GoogleNews import GoogleNews

# graph visualization
from pyvis.network import Network

# show HTML in notebook
import IPython

from keybert import KeyBERT

from sentence_transformers import SentenceTransformer
import numpy as np
import nltk
import re
from typing import List, Tuple, Union

import spacy

import inspect
import os

import ollama
from tqdm import tqdm
import time
import re
import sys
import pandas as pd

module_path = '../core'
sys.path.append(module_path)

from SemanticSimilarity import SemanticSimilarity
from KeywordExtractor import KeywordExtractor
from TripletGenerator import TripletGenerator

/home/niamatzawad/miniconda3/envs/phd/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.max_rows', None)     # Display all rows


In [3]:
class KG:
    def __init__(self,):
        self.triplets = []

    def add_triplets(self, triplets : List):
        self.triplets.extend(triplets)


In [4]:
def retrieve_text_from_filepath(filepath):
    with open(filepath,'r') as f:
        t = f.read()
        return t


In [5]:
%%time
# folder_path = "/home/niamatzawad/niamatzawad/Datasets/UTDBox/demoNY/unsplitted"
folder_path = "/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4"

s_demo = SemanticSimilarity()
kwe_demo = KeywordExtractor()
kg_demo = KG()

folder_objects = os.listdir(folder_path)
for i, filename in enumerate(folder_objects):
    if os.path.isfile(os.path.join(folder_path, filename)):
        with open(os.path.join(folder_path, filename), 'r') as file:
            text = file.read()
            text_splitted = [text[i * 1900:(i+1) *1900] for i in range(len(text)//1900)]
            start_text_time = time.time()
            for text_s in tqdm(text_splitted):
                start_subtext_time = time.time()
                try:
                    named_entities = kwe_demo.extract_named_entities(text_s)
                except Exception as e:
                    print(e)
                    continue
                s_demo.generate_reference(named_entities)
                t = TripletGenerator(text_s, s_demo, kwe_demo, os.path.join(folder_path, filename), named_entities=named_entities)
                triplets = t.get_triplets()
                kg_demo.add_triplets(triplets)
                print(f"Time to create triplets for current subtext - {time.time() - start_subtext_time}")
                
        print(f"Finished processing {i+1}/ {len(folder_objects)} files")
        print(f"Total time taken for current text - {time.time() - start_text_time}")
        print("---")
        


  0%|                                                     | 0/2 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


the text is 
The Ritz-Carlton Jakarta is a hotel and skyscraper in Jakarta, Indonesia and 14th Tallest building in Jakarta.  It is located in city center of Jakarta, near Mega Kuningan, adjacent to the sister JW Marriott Hotel.  It is operated by The Ritz-Carlton Hotel Company.  The complex has two towers that comprises a hotel and the Airlangga Apartment respectively.  The hotel was opened in 2005./nThe Oberoi family is an Indian family that is famous for its involvement in hotels, namely through The Oberoi Group./nIshqbaaaz (English: "Lovers") is an Indian drama television series which is broadcast on Star Plus.  It premiered on 27 June 2016 and airs Mon-Fri 10-11pm IST. Nakuul Mehta, Kunal Jaisingh and Leenesh Mattoo respectively portray Shivaay, Omkara and Rudra, the three heirs of the Oberoi family.  The show initially focused on the tale of three brothers, later become centered on the love story of Shivaay and Annika (Surbhi Chandna); with the story of Omkara and Rudra being shif

 50%|██████████████████████▌                      | 1/2 [00:06<00:06,  6.30s/it]

the triplets are
[('the ritz - carlton jakarta', ' operates', 'the ritz - carlton jakarta', 'The Ritz-Carlton Jakarta is operated by The Ritz-Carlton Hotel Company.'), ('the ritz - carlton jakarta', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_1.txt'), ('nthe oberoi family', ' is famous for', 'hotel', 'The Oberoi family is famous for its involvement in hotels, namely through The Oberoi Group.'), ('nthe oberoi family', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_1.txt'), ('star plus', ' broadcasts', ' Ishqbaaaz', 'Ishqbaaaz is broadcast on Star Plus.'), ('star plus', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_1.txt'), ('nakuul mehta', ' portrays', 'shivaay', 'Nakuul Mehta portrays Shivaay in Ishqbaaaz.'), ('nakuul mehta', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_1.txt'), ('the eppley hotel company', ' built', 'hotel', 'The Hotel Tallcorn was built by the Eppley Hote

100%|█████████████████████████████████████████████| 2/2 [00:13<00:00,  6.64s/it]


...

the text is 
Radio City is India's first private FM radio station and was started on 3 July 2001.  It broadcasts on 91.1 (earlier 91.0 in most cities) megahertz from Mumbai (where it was started in 2004), Bengaluru (started first in 2001), Lucknow and New Delhi (since 2003).  It plays Hindi, English and regional songs.  It was launched in Hyderabad in March 2006, in Chennai on 7 July 2006 and in Visakhapatnam October 2007.  Radio City recently forayed into New Media in May 2008 with the launch of a music portal - PlanetRadiocity.com that offers music related news, videos, songs, and other music-related features.  The Radio station currently plays a mix of Hindi and Regional music.  Abraham Thomas is the CEO of the company./nFootball in Albania existed before the Albanian Football Federation (FSHF) was created.  This was evidenced by the team's registration at the Balkan Cup tournament during 1929-1931, which started in 1929 (although Albania eventually had pressure from the teams becau

 50%|██████████████████████▌                      | 1/2 [00:05<00:05,  5.00s/it]

the triplets are
[('radio city', ' was started in', 'mumbai'), ('radio city', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_0.txt'), ('radio city', ' was started in', 'india'), ('radio city', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_0.txt'), ('radio city', ' was started in', 'hyderabad'), ('radio city', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_0.txt'), ('radio city', ' was started in', 'mumbai'), ('radio city', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_0.txt'), ('abraham thomas', ' is the CEO of', 'the sheraton corporation'), ('abraham thomas', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_0.txt'), ('albania', ' was started in', 'albania'), ('albania', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_0.txt'), ('fshf team', ' registered at', ' the Balkan Cup tournament during 1929-1931'), ('fshf team', 'filepat

100%|█████████████████████████████████████████████| 2/2 [00:10<00:00,  5.36s/it]

the triplets are
[('the song', ' owns', 'warner bros records', "Many started first as girls' seminaries or academies."), ('the song', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_0.txt'), ('salem college south', ' was started in', "girls ' seminaries", 'Salem College is the oldest female educational institution in the South.'), ('salem college south', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_0.txt'), ('salem college south', ' is the oldest', 'salem college south', 'Wesleyan College is the first that was established specifically as a college for women.'), ('salem college south', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_0.txt'), ('salem college south', ' was established specifically as', 'women', 'Generals Kiev was the only team that participated in the league the previous season.'), ('salem college south', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_0.txt'), ('ukr

In [6]:
def get_subjects_from_triplets(triplets):
    return list(set([i[0] for i in triplets]))
get_subjects_from_triplets(kg_demo.triplets)

['e wechsler',
 'milhouse',
 'newspaper',
 'lisa marie simpson',
 'shivaay',
 'roening',
 'salem college south',
 'the ritz - carlton jakarta',
 'the eppley hotel company',
 'echosmith',
 'local citizens',
 'albania',
 'hk kremenchukfirst for women',
 'Vowell',
 'nthe oberoi family',
 'faber',
 'ukrainian',
 'goertz',
 'matt groening',
 'arthur county courthouse',
 'the song',
 'r jay simpson',
 't s . arthur',
 'james l brooks',
 'radio city',
 'the simpsons',
 'john ortved',
 'dan sallit']

In [7]:
kg_demo.triplets

[('the ritz - carlton jakarta',
  ' operates',
  'the ritz - carlton jakarta',
  'The Ritz-Carlton Jakarta is operated by The Ritz-Carlton Hotel Company.'),
 ('the ritz - carlton jakarta',
  'filepath',
  '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_1.txt'),
 ('nthe oberoi family',
  ' is famous for',
  'hotel',
  'The Oberoi family is famous for its involvement in hotels, namely through The Oberoi Group.'),
 ('nthe oberoi family',
  'filepath',
  '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_1.txt'),
 ('shivaay',
  ' portrays',
  'nthe oberoi family',
  'Nakuul Mehta respectively portrays Shivaay, one of the heirs of the Oberoi family.'),
 ('shivaay',
  'filepath',
  '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_1.txt'),
 ('the eppley hotel company',
  ' built',
  'hotel',
  'The Hotel Tallcorn was built by the Eppley Hotel Company.'),
 ('the eppley hotel company',
  'filepath',
  '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_

In [8]:
# np.save('disambiguated_reference_list_hotpot_100.npy', s.disambiguated_reference_list)    # .npy extension is added if not given
# disabiguated_reference_liit = np.load('disambiguated_reference_list.npy')

In [9]:
# np.save('embedded_reference_list_hotpot_100.npy', s.embedded_reference)    # .npy extension is added if not given
# embedded_reference = np.load('embedded_reference.npy')

In [10]:
s_demo.disambiguated_reference_list

['the ritz - carlton jakarta',
 'hotel',
 'skyscraper',
 'mega kuningan',
 'the sister',
 'the complex',
 'the airlangga apartment',
 'nthe oberoi family',
 'star plus',
 'nakuul mehta',
 'leenesh mattoo',
 'shivaay',
 'omkara',
 'rudra',
 'three',
 'annika',
 'surbhi chandna',
 'marshalltown , iowa',
 'local citizens',
 'this',
 'the eppley hotel company',
 'the sheraton corporation',
 'the tallcorn towers apartments',
 'the national register of historic places',
 'rudraey oberoi']

In [8]:
from py2neo import Graph, Node, Relationship



# Connect to the Neo4j database
graph = Graph("neo4j://localhost:7687", auth=("neo4j", "password"))
graph.delete_all()

In [9]:
from py2neo import Graph, Node, Relationship



# Connect to the Neo4j database
graph = Graph("neo4j://localhost:7687", auth=("neo4j", "password"))
graph.delete_all()

triplets = kg_demo.triplets

# Create the nodes and relationships
for t in triplets:
    # Create the subject node
    subject, predicate, obj = t[0],t[1],t[2]
    context = None
    if len(t) == 4:
        context = t[3]
    
    try:
        n = graph.run(f"MATCH (n:`{subject}`) RETURN n;").data()
    except Exception as e:
        print(f"Problem with subject {subject}")
        continue

    if n:
        subject_node = n[0]["n"]
    else:
        subject_node = Node(subject, id=subject)
        graph.create(subject_node)

    if predicate == "filepath":
        # obj = re.sub("NYT Files", "'NYT Files'",obj)
        obj = "```"+obj+"```"
        context = retrieve_text_from_filepath(obj[3:-3])
        n = graph.run(f"MATCH (n:{obj}) RETURN n;").data()
        if n:
            # print(n[0]["n"])
            object_node = n[0]["n"]
        else:
            # obj = re.sub("'NYT Files'", "NYT Files",obj)[1:-1]
            
            obj = obj[2:-2]
            object_node = Node(obj, id=obj)
            graph.create(object_node)
    else:
        try:
            n = graph.run(f"MATCH (n:`{obj}`) RETURN n;").data() #I had the assumption that subjects are being linked to wrong entities. Seems that is not the case as it exactly matches object if it exists and not the most similar object
        except Exception as e:
            print(f"Problem with object - {obj}")
            continue
            
        if n:
            object_node = n[0]["n"]
        else:
            object_node = Node(obj, id=obj, text=context)
            graph.create(object_node)

    relationship = Relationship(subject_node, predicate, object_node, text=context)
    graph.merge(relationship)

print("Triplets added to the Neo4j database.")

Triplets added to the Neo4j database.


In [13]:
s = SemanticSimilarity()
s.disambiguated_reference_list

[]

In [20]:
np.save('disambiguated_reference_list_hotpot_4.npy', s_demo.disambiguated_reference_list)    # .npy extension is added if not given
np.save('embedded_reference_kg_demo_hotpot_4.npy', s_demo.embedded_reference)    # .npy extension is added if not given


In [14]:
kg_demo.triplets

[('the ritz - carlton jakarta',
  ' operates',
  'the ritz - carlton jakarta',
  'The Ritz-Carlton Jakarta is operated by The Ritz-Carlton Hotel Company.'),
 ('the ritz - carlton jakarta',
  'filepath',
  '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_1.txt'),
 ('nthe oberoi family',
  ' is famous for',
  'hotel',
  'The Oberoi family is famous for its involvement in hotels, namely through The Oberoi Group.'),
 ('nthe oberoi family',
  'filepath',
  '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_1.txt'),
 ('shivaay',
  ' portrays',
  'nthe oberoi family',
  'Nakuul Mehta respectively portrays Shivaay, one of the heirs of the Oberoi family.'),
 ('shivaay',
  'filepath',
  '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_1.txt'),
 ('the eppley hotel company',
  ' built',
  'hotel',
  'The Hotel Tallcorn was built by the Eppley Hotel Company.'),
 ('the eppley hotel company',
  'filepath',
  '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_

In [15]:
triples = []
for k in kg_demo.triplets:
    t = list(k)
    if len(t) == 3:
        t = t + [""]
    if len(t) > 4:
        continue
    triples.append(t)



['the ritz - carlton jakarta', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_1.txt']
['nthe oberoi family', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_1.txt']
['shivaay', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_1.txt']
['the eppley hotel company', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_1.txt']
['local citizens', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_1.txt']
['lisa marie simpson', ' appeared_in', 'the simpsons']
['lisa marie simpson', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_2.txt']
['the simpsons', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_2.txt']
['the simpsons', ' created_by', 'matt groening']
['the simpsons', 'filepath', '/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_4/row_2.txt']
['matt groening', 'filepath', '/home/niamatzawad/niamatzawad/Datasets

In [19]:
np.save('hotpot_4_triplets.npy', np.array(triples))